In [24]:
import numpy as np
import cv2
import os
import re
from features_description import *
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
f2, f3, f4 = describe_features((19,19))

In [3]:
f2.shape[0] + f3.shape[0] + f4.shape[0]

63960

# Read dataset 

In [10]:
faces_path = 'face.train/train/face'
non_faces_path = 'face.train/train/non-face'
face_img_names = os.listdir(faces_path)
non_face_img_names = os.listdir(non_faces_path)
for i in range(len(face_img_names)):
    face_img_names[i] = faces_path + '/' + face_img_names[i]
for i in range(len(non_face_img_names)):
    non_face_img_names[i] = non_faces_path + '/' + non_face_img_names[i]
face_img_names = np.array(face_img_names)
non_face_img_names = np.array(non_face_img_names)

In [43]:
def get_dataset(face_names, non_face_names):
    # return np array of (img, label)
    imgs, labels = [], []
    for i in range(len(face_names)):
        img = cv2.imread(face_names[i], 0)
        img = cv2.resize(img, (19, 19))
        imgs.append(img)
        labels.append(1)
    for i in range(len(non_face_names)):
        img = cv2.imread(non_face_names[i], 0)
        img = cv2.resize(img, (19, 19))
        imgs.append(img)
        labels.append(-1)
    return np.array(imgs), np.array(labels)

dataset = get_dataset(face_img_names, non_face_img_names)
dataset_ii = getIntegralImage(dataset[0]), dataset[1]
# for i in range(10):
#     plt.imshow(dataset[0][i], cmap='gray')
#     plt.show()


In [45]:
print(dataset[0][1])
print()
print(dataset_ii[0][1])

[[ 48  92 115 129 140 153 165 184 209 225 224 215 206 193 184 176 180 163
  139]
 [ 93  97 105 108 113 131 150 170 196 222 230 230 218 199 180 184 223 238
  218]
 [100  91  77  63  55  57  71 105 147 187 209 207 184 174 173 164 174 210
  208]
 [108  90  77  61  47  42  49  65  99 152 179 189 179 178 180 192 187 191
  195]
 [114 101  74  47  32  41  45  54  85 137 184 201 173 154 131 144 186 204
  201]
 [126  96  62  57  44  73  79  55  68 131 201 205 158 129 117 126 156 203
  206]
 [131 106  85  93 114 137 120  77  69 122 199 216 167  98  83 138 150 200
  207]
 [146 135 126 124 143 153 132  84  83 121 192 224 188 130 114 169 180 210
  217]
 [157 147 164 178 177 167 142 101  95 120 191 230 214 174 171 201 221 230
  231]
 [165 146 167 188 190 171 139 104  97 121 191 231 237 234 220 230 238 242
  240]
 [167 145 161 173 169 139  99  90  92 132 195 236 233 245 250 252 253 251
  242]
 [165 139 147 152 128  90  75 101 105 143 201 232 210 194 231 250 253 252
  234]
 [156 130 125 107  78  70  6

In [21]:

# class ImageDataset(Dataset):
#     def __init__(self, img_names, labels):
#         self.img_names = img_names
#         self.labels = labels
#     def __len__(self):
#         return len(self.img_names)
#     def __getitem__(self, idx):
#         img = cv2.imread(self.img_names[idx])
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         img = cv2.resize(img, (19,19))
#         img = img / 255
#         img = img.astype(np.float32)
#         return img, self.labels[idx]

# face_labels = np.ones(face_img_names.shape[0]).astype(np.int32)
# non_face_labels = np.ones(non_face_img_names.shape[0]).astype(np.int32) * -1
# train_dataset = ImageDataset(np.concatenate((face_img_names, non_face_img_names)), np.concatenate((face_labels, non_face_labels)))

# train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

